# Reconocimiento de señales de tráfico
### Máster Universitario en Ingeniería computacional y matemática - Área de inteligencia artificial
### Antonio González Hidalgo (agonzalezhidalgo@uoc.edu)

Usando una red neuronal convolucional.

Para la correcta funcionamiento de este notebook, el notebook debe de estar estructurado de la siguiente manera:

- ./dataset
- ./dataset/info.csv          Fichero que contiene los nombres de las señales correspondientes.
- ./dataset/train/Images      Conjunto de imágenes que formarán el entrenamiento de la red neuronal.
- ./dataset/test/info.csv.    Fichero que contiene las categórias de las imágenes de test.
- ./dataset/test/Images       Conjunto de imágenes que constituirán el test.

In [1]:
import import_ipynb
import tfm_generic_functions as traffic

# https://docs.python.org/3/library/os.html
import os

# https://docs.scipy.org/doc/numpy/reference/
import numpy as np

# https://keras.io/models/model/
import keras

from keras import models

# Core Layers: https://keras.io/layers/core/
# Convolution Layers: https://keras.io/layers/convolutional/
from keras import layers

# https://keras.io/preprocessing/image/
from keras.preprocessing.image import ImageDataGenerator

# https://keras.io/callbacks/
from keras.callbacks import ModelCheckpoint

importing Jupyter notebook from tfm_generic_functions.ipynb


Using TensorFlow backend.


In [2]:
# Obtenemos el directorio actual como trabajo.
ROOT_PATH = os.getcwd()

# Establecemos la dimensión de las imágenes.
IMG_SHAPE = (64, 64)

# Configuramos la CNN
EPOCHS = [20, 25, 30]
BATCH_SIZES = [32, 64, 128]
ACTIVATION = 'elu'
OPTIMIZER = 'adam'

print("Tamaño de las imágenes de entrada: ", IMG_SHAPE)
IMG_SHAPE_LEN = IMG_SHAPE[0] * IMG_SHAPE[1]
print("Vectorizando la entrada, sería de un tamaño: ", IMG_SHAPE_LEN)

# Obtenemos los paths de trabajo
train_path = os.path.join(ROOT_PATH, "dataset_det/train")
test_path = os.path.join(ROOT_PATH, "dataset_det/test")

Tamaño de las imágenes de entrada:  (64, 64)
Vectorizando la entrada, sería de un tamaño:  4096


In [3]:
# Cargamos las imágenes de entrenamiento y de test.
images_train, labels_train = traffic.readDataset(train_path, IMG_SHAPE, False)
images_test, labels_test = traffic.readDataset(test_path, IMG_SHAPE, False)

# Convertimos las listas a array numpy de float32
np_images_train = np.asarray(images_train, dtype = np.float32)
np_labels_train = np.asarray(labels_train, dtype = np.int8)

np_images_test = np.asarray(images_test, dtype = np.float32)
np_labels_test = np.asarray(labels_test, dtype = np.int8)


# Se imprime información de los datos cargados.
traffic.print_size_dataset(images_train, labels_train, np_images_train, np_labels_train, "train")
traffic.print_size_dataset(images_test, labels_test, np_images_test, np_labels_test, "test")

Total images (train):  15481
Total labels (train):  2
Images shape:  (15481, 64, 64, 3)
Labels shape:  (15481,)
Total images (test):  14726
Total labels (test):  2
Images shape:  (14726, 64, 64, 3)
Labels shape:  (14726,)


In [4]:
# Convertimos las labels de manera categórica
labels_categorical_train = keras.utils.to_categorical(np_labels_train)
labels_categorical_test = keras.utils.to_categorical(np_labels_test)

print("Ejemplo primera imagen de manera categórica: ", labels_categorical_train[0])

Ejemplo primera imagen de manera categórica:  [1. 0.]


In [5]:
def get_keras_model(activation):
    # IMPLEMENTACIÓN RED NEURONAL
    # En Keras la envoltura para cualquier red neuronal se crea con la clase Sequential
    model = models.Sequential()

    model.add(layers.Conv2D(32, (5, 5),
                            activation=activation, input_shape=(IMG_SHAPE[0], IMG_SHAPE[1], 3)))
    model.add(layers.MaxPooling2D(2, 2))
    model.add(layers.Dropout(0.2))

    model.add(layers.Conv2D(64, (5, 5), activation=activation))
    model.add(layers.MaxPooling2D(2, 2))
    model.add(layers.Dropout(0.2))
    
    model.add(layers.Conv2D(128, (5, 5), activation=activation))
    model.add(layers.MaxPooling2D(2, 2))
    model.add(layers.Dropout(0.2))

    model.add(layers.MaxPooling2D(2, 2))
    model.add(layers.Dropout(0.2))

    model.add(layers.Flatten())
    model.add(layers.Dense(len(set(labels_train)), activation='softmax'))
    return model

In [6]:
for epoch in EPOCHS:
    for batch_size in BATCH_SIZES:
            print("Epoch: {} - Batch_Size: {}".format(epoch, batch_size))
            model = get_keras_model(ACTIVATION)
            model.compile(loss = "categorical_crossentropy",
                             optimizer = OPTIMIZER,
                             metrics = ['accuracy'])
    
            model.fit(np_images_train, labels_categorical_train,
                      verbose = 2,
                      batch_size = batch_size,
                      epochs = epoch,
                      callbacks=[ModelCheckpoint('model_64_64_det_' + ACTIVATION + '.h5', save_best_only = False)])

            test_loss, test_acc = model.evaluate(np_images_test, labels_categorical_test)
            print('Test loss:', test_loss)
            print('Test accuracy:', test_acc)
            

Epoch: 20 - Batch_Size: 32
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Epoch 1/20
 - 92s - loss: 0.1536 - acc: 0.9456
Epoch 2/20
 - 89s - loss: 0.0719 - acc: 0.9794
Epoch 3/20
 - 87s - loss: 0.0453 - acc: 0.9868
Epoch 4/20
 - 85s - loss: 0.0720 - acc: 0.9814
Epoch 5/20
 - 85s - loss: 0.0421 - acc: 0.9893
Epoch 6/20
 - 84s - loss: 0.0521 - acc: 0.9887
Epoch 7/20
 - 82s - loss: 0.0321 - acc: 0.9932
Epoch 8/20
 - 86s - loss: 0.0460 - acc: 0.9906
Epoch 9/20
 - 85s - loss: 0.0471 - acc: 0.9912
Epoch 10/20
 - 85s - loss: 0.0782 - acc: 0.9886
Epoch 11/20
 - 89s - loss: 0.0520 - acc: 0.9913
Epoch 12/20
 - 90s - loss: 0.0429 - acc: 0.9932
Epoch 13/20
 - 92s - loss: 0.0510 - acc: 0.9920
Epoch 14/20
 - 88s - loss: 0.0331 - acc: 0.9948
Epoch 15/20
 - 89s - loss: 0.0686 - acc: 0.9895
Epoch 16/20
 - 90s - loss: 

14726/14726 [==============================] - 38s 3ms/step
Test loss: 0.7619900091657942
Test accuracy: 0.8460545973108787
Epoch: 30 - Batch_Size: 32
Epoch 1/30
 - 138s - loss: 0.1871 - acc: 0.9339
Epoch 2/30
 - 138s - loss: 0.0600 - acc: 0.9818
Epoch 3/30
 - 138s - loss: 0.0510 - acc: 0.9857
Epoch 4/30
 - 138s - loss: 0.0362 - acc: 0.9889
Epoch 5/30
 - 136s - loss: 0.0605 - acc: 0.9858
Epoch 6/30
 - 136s - loss: 0.0430 - acc: 0.9894
Epoch 7/30
 - 137s - loss: 0.0384 - acc: 0.9913
Epoch 8/30
 - 136s - loss: 0.0381 - acc: 0.9924
Epoch 9/30
 - 137s - loss: 0.0423 - acc: 0.9909
Epoch 10/30
 - 135s - loss: 0.0727 - acc: 0.9886
Epoch 11/30
 - 135s - loss: 0.0493 - acc: 0.9917
Epoch 12/30
 - 135s - loss: 0.0423 - acc: 0.9926
Epoch 13/30
 - 135s - loss: 0.0177 - acc: 0.9969
Epoch 14/30
 - 135s - loss: 0.0391 - acc: 0.9938
Epoch 15/30
 - 139s - loss: 0.0341 - acc: 0.9939
Epoch 16/30
 - 140s - loss: 0.0659 - acc: 0.9912
Epoch 17/30
 - 139s - loss: 0.0523 - acc: 0.9930
Epoch 18/30
 - 139s - los